<a href="https://colab.research.google.com/github/patlewig/aim/blob/master/notebooks/Comptox_lists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Combining lists from CompTox Dashboard into a single fp file

Purpose: Create a file from multiple lists of interest (IRIS,TSCA,TOXCAST, etc..) and combine them into one fingerprint file to easily compare the feature space of the two fingerprint sets.

#### QAPP ID: I-CCED-0032994-QP-1-0
#### Author: Matthew Adams
#### Principal Investigator: Grace Patlewicz
#### Last Modified May 05 2022


In [ ]:
import pandas as pd
import glob

a_dict = {}
for file in glob.glob('*.csv'):
  df = pd.read_csv(file)
  df['DTXSID'] = df['DTXSID'].map(lambda x: x.lstrip('https://comptox.epa.gov/dashboard/chemical/details/'))
  df = df[["DTXSID", "CASRN", "SMILES", "INCHIKEY"]]
  file_short = file.split('-')[0]
  df[file_short] = 1
  a_dict[f"df_{file_short}"] = df



for i, (key,value) in enumerate(a_dict.items()):
  if i == 0:
    df1, df2 = [list(a_dict.values())[i], list(a_dict.values())[i+1]]
    df1 = pd.merge(df1, df2, on=['DTXSID', 'INCHIKEY', 'SMILES', 'CASRN'],how='outer')
  elif i < (len(a_dict) - 1):
    df2 = list(a_dict.values())[i+1]
    df1 = pd.merge(df1, df2, on=['DTXSID', 'INCHIKEY', 'SMILES', 'CASRN'],how='outer')



df1['TAG_list'] = df1.apply(lambda x: x.index[x == 1].tolist(), axis=1)
df1 = df1[["DTXSID", "CASRN", "SMILES", "INCHIKEY", "TAG_list"]]
df1.to_csv('CompTox_SUPERSET.csv')



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
## SMI File -> we will need this to get our fingerprints file
df_smiles = df1[['SMILES','DTXSID']].dropna()

df_smiles.to_csv('Comptox_SUPERSET.smi', sep="\t", index=False, header=False)

